In [0]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from scipy.io import loadmat
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


In [16]:
dt=loadmat('/content/cancer.mat')
dt

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Aug 01 17:46:29 2017',
 '__version__': '1.0',
 'cancerTypes': array([[array(['BRCA'], dtype='<U4')],
        [array(['BRCA'], dtype='<U4')],
        [array(['BRCA'], dtype='<U4')],
        ...,
        [array(['UCEC'], dtype='<U4')],
        [array(['UCEC'], dtype='<U4')],
        [array(['UCEC'], dtype='<U4')]], dtype=object),
 'data': array([[44.02354236,  9.21628619, 11.31907844, ...,  4.2956195 ,
          8.76876844,  1.        ],
        [29.74615746,  9.7656002 , 40.540128  , ...,  3.81901941,
          5.75850117,  1.        ],
        [35.79931536,  9.88478062,  3.886043  , ...,  4.38238737,
          5.30617722,  1.        ],
        ...,
        [14.66541424, 20.19564581,  6.7034771 , ...,  1.83021579,
          2.35529186,  5.        ],
        [31.11602192, 14.56206615,  9.12158501, ...,  1.17613142,
          1.89129728,  5.        ],
        [74.04662769, 13.00396207,  9.47227364

In [17]:

data=pd.DataFrame(dt['data'])
data.shape

(2086, 972)

In [0]:
data.columns=[str(i[0])  for i in np.ravel(dt['geneIds'])]+['cancerTypes']

In [19]:
data.head()

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,ACBD3,ACD,ACLY,ACOT9,ADAM10,ADAT1,ADH5,ADI1,ADO,ADRB2,AGL,AKAP8L,AKAP8,AKR7A2,AKT1,ALAS1,ALDH7A1,ALDOA,ALDOC,AMDHD2,ANKRD10,ANO10,ANXA7,APBB2,APOE,APPBP2,APP,ARFIP2,ARHGAP1,ARHGEF12,...,USP14,USP1,USP22,USP6NL,USP7,UTP14A,VAPB,VAT1,VAV3,VDAC1,VGLL4,VPS28,VPS72,WASF3,WDR61,WDR67,WDR7,WDTC1,WFS1,WIPF2,WRB,XBP1,XPNPEP1,XPO7,YKT6,YME1L1,YTHDF1,ZDHHC6,ZFP36,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10,cancerTypes
0,44.023542,9.216286,11.319078,33.215176,16.901427,9.031338,1.109961,20.017821,16.724363,10.494192,17.108008,8.389632,25.574322,12.176509,28.681112,6.907185,65.046759,18.896721,8.451952,1.280492,7.066373,17.305337,13.109840,20.162335,42.926568,14.877014,35.806714,234.617764,6.877375,3.856382,21.482055,11.967563,43.623787,22.920785,63.243092,9.367205,949.688861,23.894304,78.191770,33.708319,...,13.751053,20.503647,61.829856,8.279923,29.073110,13.168753,13.126014,40.021467,15.962893,66.313909,34.647426,42.882030,21.952810,4.441013,28.687170,2.293086,3.401144,21.784669,6.783859,8.659369,12.183537,97.219006,23.011699,15.462851,23.105338,38.696649,16.412698,14.100348,89.149907,29.275809,21.706486,16.315579,4.224009,8.602081,23.762341,8.302416,1.408731,4.295620,8.768768,1.0
1,29.746157,9.765600,40.540128,30.169134,20.047393,32.237287,2.460624,17.029112,28.346167,17.017284,23.657241,4.312751,42.834098,7.860691,100.156060,4.581611,46.213276,9.425537,11.638276,0.672496,20.530604,12.648248,9.443037,21.162685,56.971827,19.422322,15.911620,436.015850,5.033877,6.679406,23.798112,16.638117,103.988079,26.798762,91.291993,10.467921,283.907081,44.294113,87.901646,33.955580,...,15.812551,12.532382,26.868496,4.408805,34.243400,9.272388,19.199148,82.916593,41.278863,124.070088,25.091608,77.510440,37.234040,1.130839,45.035220,1.844657,3.710194,15.396392,21.020186,12.324063,20.268603,1100.969122,17.763978,18.222554,26.112132,53.158154,24.657516,15.957426,58.026493,65.896789,12.815215,10.150965,8.914809,6.797915,15.379187,11.420690,6.599729,3.819019,5.758501,1.0
2,35.799315,9.884781,3.886043,29.984211,17.135946,21.273727,1.501203,20.598204,25.855152,12.275738,34.299298,5.382395,47.948343,8.969267,29.285883,3.957447,50.324875,14.414314,7.230889,0.680324,11.658338,16.061290,10.238229,56.257662,51.789016,16.852390,23.589880,605.608766,4.426113,8.362821,16.546149,17.599537,67.416569,15.511715,56.696131,8.975012,390.612984,40.131332,131.799558,26.041611,...,18.621191,15.962939,39.981431,5.660734,40.493858,12.174465,15.395302,96.262929,57.423509,90.637427,21.923186,76.943971,42.031667,1.419108,38.380249,2.728448,3.847367,24.992626,25.796499,13.129145,22.455759,1044.965942,19.992626,23.268570,30.432389,74.463368,17.926723,13.988009,26.157333,44.571276,14.344729,11.224647,7.870991,7.724003,25.762396,8.628786,4.104879,4.382387,5.306177,1.0
3,26.490401,7.085828,10.804003,23.482255,17.044085,14.880104,1.299056,14.978582,31.214294,10.015235,18.990297,6.287945,42.671544,8.677915,22.096538,2.722100,50.500283,16.545271,9.018492,1.567676,10.772595,20.430683,13.104744,58.722716,63.681419,16.996255,25.243284,730.205265,4.723555,9.844082,25.308774,10.570067,77.792078,8.774143,106.168592,5.814626,285.281947,54.146090,102.598979,25.780804,...,15.631805,17.017821,34.311549,5.947355,35.019612,15.533345,10.490997,69.717675,95.219492,117.829916,13.202075,93.570080,37.054869,1.212296,43.809284,4.354137,2.645756,20.319431,35.401242,7.312202,12.250125,1798.786942,18.344660,13.925511,31.476416,54.208713,21.799192,22.550758,58.898222,51.223656,13.660995,9.730124,7.804760,5.030966,8.964868,7.990036,4.251886,3.702483,7.500498,1.0
4,27.632466,7.642971,3.670265,16.584843,20.375321,22.174600,1.553541,14.909150,54.435490,13.392213,15.802764,4.549983,47.859872,7.881087,28.001759,4.558604,46.746612,11.772061,6.489624,0.518419,5.001113,20.393187,10.740420,64.713959,81.127483,16.675901,20.812069,2367.284747,1.926937,25.453773,27.230046,19.039812,82.315651,32.013791,59.285915,7.037963,771.743788,58.389219,1

In [0]:
y = data['cancerTypes']

In [0]:
corr=data.corr()

In [0]:
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False
        if corr.iloc[i,j] <= 0.1:
            if columns[j]:
                columns[j] = False
        
selected_columns = data.columns[columns]
data = data[selected_columns]

In [23]:
data

,AARS,ABCB6,ABCC5,ABCF1,ABCF3
0,44.023542,9.216286,11.319078,33.215176,16.901427
1,29.746157,9.765600,40.540128,30.169134,20.047393
2,35.799315,9.884781,3.886043,29.984211,17.135946
3,26.490401,7.085828,10.804003,23.482255,17.044085
4,27.632466,7.642971,3.670265,16.584843,20.375321
5,32.046192,8.419004,14.540679,28.260823,24.671378
6,37.176767,7.449734,13.598654,26.924645,23.875332
7,71.272871,8.224724,4.675322,21.374021,17.651699
8,39.924566,2.535918,10.070192,44.952255,8.639155
9,24.747010,8.214457,9.055250,22.329043,17.008653


In [0]:
x = data

In [0]:
train_x,test_x,train_y,test_y = train_test_split(x,y, test_size = 0.2)

In [40]:
train_x.shape,train_y.shape

((1668, 5), (1668,))

In [0]:
model = SVC(kernel = 'linear')

In [28]:
model.fit(train_x,train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [29]:
# Use the forest's predict method on the test data
predictions = model.predict(test_x)
# Calculate the absolute errors
errors = abs(predictions - test_y)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.71 degrees.


In [30]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_y)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 73.44 %.


In [0]:
ans= model.predict(train_x)

In [33]:
print(ans)

[2. 1. 1. ... 1. 2. 1.]


In [0]:
from sklearn.linear_model import LogisticRegression 

In [0]:
model1 = LogisticRegression()

In [43]:
model1.fit(train_x,train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
predictions = model1.predict(test_x)

In [46]:
predictions

array([2., 1., 1., 1., 2., 2., 2., 2., 1., 1., 1., 1., 1., 5., 1., 2., 1.,
       1., 2., 1., 1., 1., 2., 4., 1., 1., 2., 2., 2., 1., 1., 2., 5., 2.,
       1., 1., 1., 2., 2., 1., 1., 1., 1., 1., 4., 1., 1., 1., 2., 1., 1.,
       2., 2., 1., 2., 1., 1., 2., 5., 2., 1., 2., 1., 1., 2., 5., 2., 1.,
       2., 2., 1., 1., 2., 1., 1., 1., 1., 2., 1., 5., 1., 1., 1., 2., 2.,
       1., 1., 1., 1., 2., 1., 2., 1., 1., 2., 5., 2., 1., 1., 1., 2., 1.,
       1., 5., 1., 2., 1., 1., 1., 5., 2., 1., 1., 1., 1., 1., 1., 5., 2.,
       1., 1., 5., 2., 4., 1., 2., 2., 2., 1., 1., 1., 2., 2., 4., 1., 5.,
       5., 1., 1., 2., 1., 1., 2., 1., 1., 1., 4., 2., 2., 1., 1., 2., 2.,
       5., 4., 1., 1., 1., 1., 1., 1., 2., 1., 1., 5., 5., 1., 2., 2., 5.,
       1., 1., 1., 1., 4., 5., 2., 1., 5., 2., 1., 1., 2., 2., 1., 2., 1.,
       2., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 5., 1., 2.,
       2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 4., 2., 1., 2., 2.,
       1., 2., 4., 5., 1.

In [47]:
errors = abs(predictions - test_y)
# Calculate mean absolute percentage error (MAPE)

mape = 100 * (errors / test_y)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 68.08 %.
